<a href="https://colab.research.google.com/github/Dileep614/Dileep-Kumar-konakalla/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FOUNDATIONS OF MODERN MACHINE LEARNING, IIIT Hyderabad**
### MODULE: CLASSIFICATION-1
### LAB-3 : Using KNN for Text Classification
#### Module Coordinator: Jashn Arora


---

## **Section 1: Understanding NLP tools**

In this lab we will be using KNN on a real world NLP application i.e. is text classification. But first look at some NLP techniques for text classification and tools that we use when we want to use python for NLP.

## Section 1.2: Data Cleaning and Preprocessing step

Raw text must be processed and converted into a form so that it is suitable to use with various machine-learning algorithms.  
In case of text, there are lots of things that need to be taken into account.  


1.   Removing numbers from the text
2.   Handling capitalization and punctuation.
3.   Stemming and Lemmatizing text.  

And most importantly, one can't just use words or images directly in algorithms; they need to be converted into vectors- a form that algorithms can understand.



### **NLTK**
NLTK (or Natural Language Tool Kit) is a commonly used library for processing text. We will use this tool in this lab. Lets first install it.


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
import re
import numpy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

def cleanText(text, lemmatize, stemmer):
    """Method for cleaning text from train and test data. Removes numbers, punctuation, and capitalization. Stems or lemmatizes text."""

    if isinstance(text, float):
        text = str(text)
    if isinstance(text, numpy.int64):
        text = str(text)
    try:
        text = text.decode()
    except AttributeError:
        pass

    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    text = re.sub(r"[^A-Za-z]", " ", text)
    text = text.lower()


    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()

        def get_tag(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return ''

        text_result = []
        tokens = word_tokenize(text)  # Generate list of tokens
        tagged = pos_tag(tokens)
        for t in tagged:
            try:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0], get_tag(t[1][:2])))
            except:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0]))
        return text_result

    if stemmer:
        text_result = []
        tokens = word_tokenize(text)
        snowball_stemmer = SnowballStemmer('english')
        for t in tokens:
            text_result.append(snowball_stemmer.stem(t))
        return text_result

In [ ]:
sample_text = "Troubling"
sample_text_result = cleanText(sample_text, lemmatize=False, stemmer=True)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text)
print(sample_text_result)
sample_text_result = cleanText(sample_text, lemmatize=True, stemmer=False)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text_result)

Troubling
troubl
trouble


## Section 1.2: BAG OF WORDS

A bag-of-words model, or BoW for short, is a way of extracting features from text for use in modeling, such as with machine learning algorithms.

The approach is very simple and flexible, and can be used in many ways for extracting features from documents.

A bag-of-words is a representation of text that describes the occurrence of words within a document.
It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.

In [ ]:
# Functions to convert document(s) to a list of words, with the option of removing stopwords. Returns document-term matrix.

def createBagOfWords(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = CountVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer = CountVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    bag_of_words_train = vectorizer.fit_transform(clean_train).toarray()
    bag_of_words_test = vectorizer.transform(clean_test).toarray()
    return bag_of_words_train, bag_of_words_test


## Section 1.3: TF-IDF
TF-IDF technique is used to find meaning of sentences consisting of words and cancels out the incapabilities of Bag of Words technique which is good for text classification or for helping a machine read words in numbers.

The number of times a term occurs in a document is called its Term frequency (TF).

 Document frequency is the number of documents in which the word is present.  Inverse DF (IDF) is the inverse of the document frequency which measures the informativeness of term *t*.




In [ ]:
def createTFIDF(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = TfidfVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer =  TfidfVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    tfidf_train = vectorizer.fit_transform(clean_train).toarray()
    tfidf_test = vectorizer.transform(clean_test).toarray()
    return tfidf_train, tfidf_test

# **Section 2: UNDERSTANDING THE DATA : A REVIEWS DATASET**

Sentiment analysis is the interpretation and classification of emotions (such as positive, negative and neutral) within text data using text analysis techniques.  
Given below is a dataset consisting of reviews along with sentiment class (positive or negative).

In [ ]:
# Upload the Reviews CSV file that has been shared with you.
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving reviews.csv to reviews.csv


In [ ]:
import pandas as pd
df = pd.read_csv('reviews.csv')

In [ ]:
df = df.dropna()

In [ ]:
df.to_csv('reviews.csv', index=False)

# **Section 3: KNN MODEL**

Given below are two KNN models; in the first case we are using Bag-of-Words and in the second case we are using TF-IDF.
Note the different metrics and parameters used in each.

In [ ]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 1: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"],
                                                        test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2,
                                         metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

Note: Cross-validation will be discussed in detail in the upcoming lab session.

In [ ]:
## KNN accuracy after using BoW
predicted, y_test = bow_knn()

<ipython-input-2-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BOW accuracy = 62.30366492146597%


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.62
[0.60784314 0.58431373 0.66141732]




In [ ]:
## KNN accuracy after using TFIDF
predicted, y_test = tfidf_knn()

<ipython-input-2-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TFIDF accuracy = 70.15706806282722%


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Cross Validation Accuracy: 0.73
[0.7254902  0.74117647 0.72834646]


# Section 4: SPAM TEXT DATASET
Now let's use what we've learnt to classify texts as spam or not spam.

In [ ]:
# Upload the spam text data CSV file that has been shared with you. You can also download the file from https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam.csv



ChatGPT
You said:
Which is the best combination of features?
How would you test or visualize four or more features?
Can you come up with your own features?
Will these features work for different classes other than 0 and 1?
What will happen if we take more that two classes at a time?
ChatGPT said:
You're asking a set of questions related to feature selection, visualization, and classification with multiple classes. Let's go through them one by one.
1. Which is the best combination of features?The best combination of features depends on the dataset and the problem at hand. You can determine this using:Feature Selection Techniques:Filter Methods (e.g., correlation, mutual information)Wrapper Methods (e.g., Recursive Feature Elimination, forward/backward selection)Embedded Methods (e.g., Lasso regression, decision tree feature importance)Dimensionality Reduction:PCA (Principal Component Analysis): Reduces feature space while preserving variance.LDA (Linear Discriminant Analysis): Useful for classification problems.Model Performance:Train models with different feature sets and evaluate using accuracy, F1-score, AUC-ROC, etc.2. How would you test or visualize four or more features?For visualization, use:Pair plots: Scatterplots for different feature combinations.Heatmaps: To check feature correlations.3D plots: If using three features (e.g., Matplotlib's plot_trisurf).Parallel Coordinates Plot: Shows high-dimensional data trends.T-SNE / UMAP: Reduce dimensions to 2D or 3D for visualization.For testing, train a machine learning model (e.g., logistic regression, decision tree) and evaluate its performance.3. Can you come up with your own features?Yes! Feature engineering is key to improving models. Some techniques include:Creating ratios or interactions (e.g., feature1/feature2, feature1 * feature3).Extracting features from text (e.g., TF-IDF, word embeddings).Generating temporal features (e.g., time of day, day of the week).Using domain knowledge to transform data (e.g., log transformation, polynomial features).4. Will these features work for different classes other than 0 and 1?If the features were designed for binary classification (0 and 1), they might not generalize well to multi-class problems. You should:Check whether features separate different classes effectively.Use methods like One-vs-Rest (OvR) or One-vs-One (OvO) for classification.Consider feature importance for each class.5. What will happen if we take more than two classes at a time?The problem shifts from binary classification to multi-class classification. Some effects:More complex decision boundaries: Models like logistic regression and SVM need adaptations (softmax, OvR, OvO).Different evaluation metrics: Accuracy, precision, recall, and F1-score must be computed per class.Feature importance might change: Some features might be useful for certain classes but not others.Need for advanced models: Decision trees, neural networks, or ensemble methods might be required.
Would you like me to test feature selection or visualization for a dataset you have?
You said:
How many components do you need to retain 90% of the variance in the data?
Find one other use case that makes use of PCA and summarize how it is applied to the problem domain.
ChatGPT said:
1. How many components do you need to retain 90% of the variance in the data?
To determine the number of Principal Components (PCs) needed to retain 90% of the variance, follow these steps:
Standardize the Data: Ensure all features have mean = 0 and variance = 1.Compute the Covariance Matrix: This shows relationships between variables.Perform Eigenvalue Decomposition: Extract eigenvalues (variance explained by each PC).Compute the Cumulative Variance:Sum the eigenvalues.Compute the cumulative variance percentage for each component.Identify the minimum number of components needed to reach 90% variance.
Would you like me to perform this analysis on a dataset you provide?
2. Another Use Case for PCA
Use Case: Facial Recognition
Application: PCA is widely used in face recognition systems to reduce the dimensionality of image data while preserving key patterns.How It Works:Convert facial images into grayscale (reduces dimensionality).Flatten images into feature vectors.Apply PCA to extract the most important features (Eigenfaces).Use these features for classification (e.g., with an SVM or neural network).Benefit: Reduces computational cost while maintaining accuracy in recognizing faces.
Would you like a code example for PCA on an image dataset?
You said:
How do you think would the number of neighbors effect the Isomap algorithm? What happens when the number of neighbors considered is very large? What happens when it is very low?

When is the ISOMAP algorithm superior to PCA?

Search up another manifold learning methods other than ISOMAP? If there are any, whats the key difference ?

Suggest ways to deal with missing data in manifold learning.
ChatGPT said:
1. Effect of Number of Neighbors in the ISOMAP Algorithm
ISOMAP (Isometric Mapping) constructs a neighborhood graph and computes geodesic distances between points to perform dimensionality reduction. The number of neighbors (k) significantly affects its performance:
When k is too large:
The graph approaches a fully connected structure, making ISOMAP behave more like classical MDS (Multidimensional Scaling).It loses the ability to capture the underlying nonlinear structure.Shortcuts may appear in geodesic distances, distorting the manifold.
When k is too small:
The graph becomes disconnected, causing errors in distance calculations.Some points might get isolated, leading to fragmented representations.Manifold structure may not be properly captured.2. When is ISOMAP Superior to PCA?
ISOMAP is better than PCA when dealing with nonlinear manifolds, such as:
Swiss Roll Data: PCA would flatten it incorrectly, while ISOMAP preserves the underlying shape.Nonlinear Data Structures: When data lies on a curved, low-dimensional manifold in high-dimensional space.When Geodesic Distances Matter: ISOMAP approximates the actual distances better than PCA, which assumes linear relationships.
However, ISOMAP is sensitive to noise and requires careful tuning of the neighborhood size.
3. Other Manifold Learning Methods (Besides ISOMAP)
One of the most widely used alternatives is t-SNE (t-Distributed Stochastic Neighbor Embedding).
Key Differences:
ISOMAP: Uses geodesic distances and preserves global structure.t-SNE: Focuses on local structure by converting pairwise distances into probabilities, making it better for clustering visualization.UMAP (Uniform Manifold Approximation and Projection): Faster than t-SNE, preserves both local and global structure better.
Would you like a comparison between ISOMAP, t-SNE, and UMAP on a dataset?
4. Handling Missing Data in Manifold Learning
Missing data can distort manifold learning. Here are some strategies to handle it:
Interpolation & Imputation
Mean/median imputation for small gaps.k-NN imputation (use nearby points to estimate missing values).Matrix factorization (e.g., PCA-based imputation).
Use Autoencoders
Train a neural network to reconstruct missing values based on learned representations.
Remove Incomplete Data Points
Only if the missing rate is low to avoid losing too much information.
Graph-Based Approaches
Construct neighborhood graphs only with available data and approximate missing values using neighbors.
Would you like an implementation of ISOMAP with missing data handling?
You said:

Lab4.ipynb
Lab4.ipynb_
Module 2: Appreciating, Interpreting and Visualizing Data
Lab 4: t-Distributed Stochastic Neighbor Embedding (t-SNE)
Coordinator: Aswin Jose
This lab will be based upon t-SNE which is a dimensionality reduction algorithm used to visualize high dimensional datasets.

t-SNE stands for t-Distributed Stochastic Neighbor Embedding. It is an unsupervised, non-linear technique primarily used for data exploration and visualizing high-dimensional data. It was developed by Laurens van der Maatens and Geoffrey Hinton in 2008 (Link to the paper: https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf)

image.png

t-SNE has a tuneable parameter, perplexity which balances attention between the local and global aspects of your data. It is a guess about the number of close neighbors each point has. The perplexity value has a complex effect on the resulting pictures. The original paper says, “The performance of t-SNE is fairly robust to changes in the perplexity, and typical values are between 5 and 50.” But the story is more nuanced than that. Getting the most from t-SNE may mean analyzing multiple plots with different perplexities.

Also the t-SNE algorithm doesn’t always produce similar output on successive runs as there are additional hyperparameters related to the optimization process.

HOW DOES T-SNE WORK??
The t-SNE algorithm calculates a similarity measure between pairs of instances in the high dimensional space and in the low dimensional space. It then tries to optimize these two similarity measures using a cost function. Let’s break that down into 3 basic steps.

Measure similarities between points in the high dimensional space. Think of a bunch of data points scattered on a 2D space. For each data point (xi) we’ll center a Gaussian distribution over that point. Then we measure the density of all points (xj) under that Gaussian distribution. Then renormalize for all points. This gives us a set of probabilities (Pij) for all points. Those probabilities are proportional to the similarities. All that means is, if data points x1 and x2 have equal values under this gaussian circle then their proportions and similarities are equal and hence you have local similarities in the structure of this high-dimensional space. The Gaussian distribution or circle can be manipulated using what’s called perplexity, which influences the variance of the distribution (circle size) and essentially the number of nearest neighbors.

This step is similar to step 1, but instead of using a Gaussian distribution we use a Student t-distribution with one degree of freedom, which is also known as the Cauchy distribution (See fig below). This gives us a second set of probabilities (Qij) in the low dimensional space. As you can see the Student t-distribution has heavier tails than the normal distribution. The heavy tails allow for better modeling of far apart distances.

The last step is that we want these set of probabilities from the low-dimensional space (Qij) to reflect those of the high dimensional space (Pij) as best as possible. We want the two map structures to be similar. We measure the difference between the probability distributions of the two-dimensional spaces using Kullback-Liebler divergence (KL). Finally, we use gradient descent to minimize our KL cost function.

students_t.jpeg

Scikit-learn has an implementation of t-SNE available which provides a wide variety of tuning parameters for t-SNE, and the most notable ones are:

n_components (default: 2): Dimension of the embedded space.
perplexity (default: 30): The perplexity is related to the number of nearest neighbors that are used in other manifold learning algorithms. Consider selecting a value between 5 and 50.
n_iter (default: 1000): Maximum number of iterations for the optimization. Should be at least 250.
method (default: ‘barnes_hut’): Barnes-Hut approximation runs in O(NlogN) time. method=’exact’ will run on the slower, but exact, algorithm in O(N^2) time.

[ ]
# numpy.
import numpy as np
from numpy import linalg
from numpy.linalg import norm
from scipy.spatial.distance import squareform, pdist

# sklearn.
import sklearn
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale

# Random state.
RS = 20150101

# matplotlib.
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib
%matplotlib inline

# seaborn.
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})
1797 images each of size 8 x 8 loaded using load_digits()


[ ]
digits = load_digits()
digits.data.shape
(1797, 64)
Printing some images from the dataset


[ ]
nrows, ncols = 2, 5
plt.figure(figsize=(6,3))
plt.gray()
for i in range(ncols * nrows):
    ax = plt.subplot(nrows, ncols, i + 1)
    ax.matshow(digits.images[i,...])
    plt.xticks([]); plt.yticks([])
    plt.title(digits.target[i])


[ ]
# We first reorder the data points according to the handwritten numbers.
X = np.vstack([digits.data[digits.target==i]
               for i in range(10)])
y = np.hstack([digits.target[digits.target==i]
               for i in range(10)])
Now using TSNE to fit the dataset with the default values.
n_components : 2
perplexity : 30
n_iter : 1000
method : ‘barnes_hut’

[ ]
digits_proj = TSNE(init="pca", random_state=RS).fit_transform(X)
Visualizing the data in the projected space

[ ]
def scatter(x, colors):
    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 10))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[colors.astype(int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(10):
        # Position of each label.
        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)
    plt.show()

    return f, ax, sc

scatter(digits_proj, y)

Tweaking some of the hyperparameters to better understand their role
Changing the PERPLEXITY values
image.png

With perplexity values in the range (5 - 50) suggested by van der Maaten & Hinton, the diagrams do show these clusters, although with very different shapes. Outside that range, things get a little weird. With perplexity 2, local variations dominate. The image for perplexity 100, with merged clusters, illustrates a pitfall: for the algorithm to operate properly, the perplexity really should be smaller than the number of points. Implementations can give unexpected behavior otherwise.

n_components : 2
perplexity : 5
n_iter : 1000
method : ‘barnes_hut’

[ ]
digits_proj = TSNE(init="pca", random_state=RS, perplexity=5).fit_transform(X)

scatter(digits_proj, y)

We can see that there are local clusters within the same number group as well. This is happening as the perplexity being at 5, allows the local neighbourhood to dominate. Let us now see what happens if we increase the perplexity to 100, thereby increasing global impact.

n_components : 2
perplexity : 100
n_iter : 1000
method : ‘barnes_hut’

[ ]
digits_proj = TSNE(init="pca", random_state=RS, perplexity=100).fit_transform(X)

scatter(digits_proj, y)

The general structure of the plot remained similar to the one with perplexity = 30 (default), but on careful observation you can observe many data points not being part of the group they are supposed to be in. This is because of the large number of points considered for the neighbourhood (as perplexity value = 100 is higher), thereby allowing 2 data points from different groups to end up closer.

Changing the NUMBER OF ITERATIONS
image.png

The images above show five different runs at perplexity 30. The first four were stopped before stability. After 10, 20, 60, and 120 steps you can see layouts with seeming 1-dimensional and even pointlike images of the clusters. If you see a t-SNE plot with strange “pinched” shapes, chances are the process was stopped too early. Unfortunately, there’s no fixed number of steps that yields a stable result. Different data sets can require different numbers of iterations to converge.

The most important thing is to iterate until reaching a stable configuration.

n_components : 2
perplexity : 30
n_iter : 250
method : ‘barnes_hut’

[ ]
digits_proj = TSNE(init="pca", random_state=RS, n_iter=250).fit_transform(X)

scatter(digits_proj, y)

As can be seen from the figure above, stopping the optimization earlier (in 250 iterations) resulted in a suboptimal clustering of the groups.

Let us now see how the results are affected if t-SNE is run for larger number of iterations

n_components : 2
perplexity : 30
n_iter : 5000
method : ‘barnes_hut’

[ ]
digits_proj = TSNE(init="pca", random_state=RS, n_iter=5000).fit_transform(X)

scatter(digits_proj, y)

Running for larger number of iterations more or less resulted in the same plot as the optimization had nearly converged till the default 1000 iterations. However the density of the clusters has increased.

Changing the method to EXACT
The barnes-hut method takes O(NlogN) time, whereas the exact method takes O(N^2) time. Notice the increase in the execution time while running the cell below.

n_components : 2
perplexity : 30
n_iter : 1000
method : ‘exact’

[ ]
digits_proj = TSNE(init="pca", random_state=RS, method='exact').fit_transform(X)

scatter(digits_proj, y)

EXERCISE
Try out different perplexity and iteration values to better appreciate the concepts taught.

You can modify the code in the cell below to check the resultant plots.


[ ]
###############################################################
###### MODIFY THE VALUES FOR THE HYPERPARAMETERS BELOW ########

perplexity_value = 2
number_iterations = 1000
method = "barnes_hut"

###############################################################

digits_proj = TSNE(init="pca", random_state=RS, n_iter=number_iterations, perplexity=perplexity_value, method=method).fit_transform(X)
scatter(digits_proj, y)

Exercise:
Now that you understand t-SNE a bit better, can you point out some differences between PCA and t-SNE. What are the advantages/disadvantages of one over the other?
Pointers:
a. Which of the two algorithms is linear and which one is non-linear?
b. How does the non-linearity in one of these two algorithms help in capturing certain data sets?
c. PCA is known to keep points which were further apart in the higher dimension, far apart in the lower dimension as well. Does t-SNE do the same? Or does it try to preserve local neighbourhood?
d. Can you comment on which one of the two is computationally more expensive?

How does the computational complexity and runtime of t-SNE scale with dataset size and dimensionality?

What are some limitations or potential pitfalls to be aware of when using t-SNE? (tell atleast
ChatGPT said:
Differences Between PCA and t-SNEFeaturePCAt-SNETypeLinearNon-linearPreservesGlobal structure (variance)Local structure (neighborhoods)ScalabilityComputationally efficient (O(n^2) for eigen decomposition)Computationally expensive (O(N log N) for Barnes-Hut, O(N²) for exact)InterpretabilityProvides principal componentsNo explicit axes or componentsOutput StabilityDeterministicCan vary between runsBest ForUnderstanding variance, feature reductionClustering and visualization of high-dimensional dataAdvantages and Disadvantages
PCA Advantages:
Fast and scalable.Works well when data follows a linear structure.Keeps distant points far apart.
PCA Disadvantages:
Does not capture non-linear relationships.May not be ideal for visualizing complex structures.
t-SNE Advantages:
Preserves local neighborhood structure.Good for high-dimensional visualization.Handles non-linearity effectively.
t-SNE Disadvantages:
Computationally expensive.Sensitive to hyperparameters (perplexity, iterations).Cannot be used for general feature reduction.Computational Complexity and Scaling of t-SNEMethod: Barnes-Hut t-SNE (default in scikit-learn)Time Complexity: O(Nlog⁡N)O(NlogN), where N is the number of points.Memory Complexity: O(N2)O(N2) (especially for large datasets).Method: Exact t-SNETime Complexity: O(N2)O(N2), making it infeasible for large datasets.Memory Complexity: Also O(N2)O(N2).
As dataset size and dimensionality increase:
Computational time grows significantly.Memory usage becomes a bottleneck.Barnes-Hut approximation is preferred for large datasets.Limitations and Pitfalls of t-SNE
Parameter Sensitivity
Perplexity greatly affects the results.Different perplexity values produce different visualizations.
Instability in Results
Runs with different initializations may yield different embeddings.
Not Good for Feature Reduction
Unlike PCA, t-SNE is not meant to reduce dimensions for downstream ML tasks.
Does Not Preserve Global Structure
Clusters might be meaningful locally but distances between clusters can be misleading.
High Computational Cost
Expensive for large datasets (compared to PCA or UMAP).
Would you like to test different hyperparameters on a dataset and analyze the results?
You said:
In the section on Decision boundaries, you must have seen that we ran the KNN algorithm twice: first with the weights set to 'uniform' and then set to 'distance'. Find out the difference between these two.
What do you think could be the drawbacks of using KNN ?
ChatGPT said:
Difference Between weights='uniform' and weights='distance' in KNN
When using k-Nearest Neighbors (KNN), the choice of weights determines how the nearest neighbors contribute to classification or regression:
weights='uniform'
Each neighbor has equal importance, regardless of how close or far it is.The class label is determined by a majority vote among k neighbors.Works well when data is evenly distributed and noise-free.
weights='distance'
Closer neighbors have higher influence than farther ones.Weights are assigned as 1/distance, meaning closer points have a greater impact.Works better when there is high variance in density or when some neighbors are more informative than others.Key Differenceuniform: Treats all neighbors equally.distance: Prioritizes closer neighbors more.If the data has irregular density, distance weighting may perform better by reducing the impact of distant, possibly irrelevant points.Drawbacks of Using KNN
While KNN is a simple and effective algorithm, it has several limitations:
Computational Cost (Slow for Large Datasets)
KNN requires storing and comparing all training data during inference.Searching for k neighbors takes O(N) time for each prediction.Can be improved with KD-trees or Ball-trees but still scales poorly.
Memory-Intensive
Unlike parametric models, KNN stores all training data, leading to high memory usage.
Sensitive to Noise and Outliers
KNN does not learn patterns—it simply retrieves nearest neighbors.A single misclassified or noisy point can strongly affect predictions.
Choice of k is Crucial
Too small → Model is too sensitive (overfitting).Too large → Model becomes too generalized (underfitting).
Curse of Dimensionality
As dimensionality increases, distance metrics become less meaningful.Most points tend to be equidistant in high-dimensional space, making KNN less effective.
Feature Scaling is Necessary
KNN relies on distance calculations (e.g., Euclidean).Different feature scales can distort results, requiring normalization (e.g., MinMax or StandardScaler).
Would you like a demonstration comparing KNN with different weight settings on a dataset?
You said:
Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
Can you think of techniques that are better than both BoW and TF-IDF ?
Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.
Finished thinking.
Why Does TF-IDF Perform Better Than Bag-of-Words (BoW)?
TF-IDF (Term Frequency-Inverse Document Frequency) usually achieves better accuracy than BoW because:
Reduces the Impact of Common Words
In BoW, frequent words (e.g., "the," "is," "and") have high importance, even if they don't contribute much to meaning.TF-IDF lowers the weight of common words by using Inverse Document Frequency (IDF), ensuring more meaningful words get higher importance.
Better Differentiation Between Important and Unimportant Words
BoW treats all words equally, while TF-IDF prioritizes rare but informative words.For example, in a medical dataset, words like "disease" or "cancer" get more importance than generic words like "patient" or "doctor."
Handles Large Vocabulary More Efficiently
BoW results in sparse matrices with many zero values.TF-IDF gives non-zero weights only to words that are both frequent and important, improving efficiency.
Reduces Dimensionality Without Losing Important Information
BoW results in a huge feature space, while TF-IDF helps by reducing the importance of redundant words, making models more generalizable.Better Techniques Than BoW and TF-IDF
Several advanced NLP techniques outperform both BoW and TF-IDF:
Word Embeddings (Word2Vec, GloVe, FastText)
How it works: Learns vector representations of words based on context.Advantages: Captures semantic meaning (e.g., "king" - "man" + "woman" ≈ "queen").Disadvantages: Requires large corpora for training and is more complex.
Transformers (BERT, GPT, T5, etc.)
How it works: Uses deep learning to understand context dynamically.Advantages: Best for text classification, summarization, and sentiment analysis.Disadvantages: Computationally expensive.
Latent Semantic Analysis (LSA) & Latent Dirichlet Allocation (LDA)
How it works: Finds topics and hidden structures in text.Advantages: Useful for topic modeling.Disadvantages: Doesn’t capture word order as well as embeddings.
Would you like a practical example comparing TF-IDF with Word Embeddings?
Stemming vs. Lemmatization
Both are techniques for text normalization, but they work differently.
FeatureStemmingLemmatizationDefinitionRemoves suffixes to get root formConverts words to their base/dictionary formMethodUses simple rules (e.g., "running" → "run")Uses a dictionary (e.g., "better" → "good")AccuracyLess accurate, more aggressiveMore accurate, linguistically correctComputational CostFasterSlower, requires more processingExample"Caring" → "Car""Caring" → "Care"
When to Use Which?
Use Stemming when speed matters and minor inaccuracies are acceptable.Use Lemmatization when you need proper words with correct meanings (e.g., NLP applications like chatbots, translation).
Would you like a code demo comparing stemming and lemmatization?
You said:
Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
Can you think of techniques that are better than both BoW and TF-IDF ?
Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.
ChatGPT said:
Why Does TF-IDF Perform Better Than Bag-of-Words (BoW)?
TF-IDF (Term Frequency-Inverse Document Frequency) improves upon BoW because it assigns weights to words based on importance, rather than just raw frequency.
Handles Common Words Better
BoW treats all words equally, meaning frequent but unimportant words (e.g., "the", "is", "and") can dominate the representation.TF-IDF reduces the weight of common words by penalizing terms that appear in many documents.
Gives More Importance to Informative Words
TF-IDF assigns higher weights to words that appear frequently in one document but rarely in others.This makes it better for distinguishing between documents.
Reduces Feature Space Sparsity
BoW results in high-dimensional sparse matrices, while TF-IDF reduces noise by lowering weights for non-informative words.This improves model efficiency and generalization.
Better Performance in Classification & Retrieval
Since TF-IDF prioritizes discriminative words, it improves text classification, clustering, and information retrieval tasks.Better Techniques Than BoW and TF-IDF
Several advanced NLP techniques outperform both BoW and TF-IDF by capturing semantic meaning and contextual relationships between words:
Word Embeddings (Word2Vec, GloVe, FastText)
How it works: Words are mapped to dense, continuous vector spaces based on their meanings and context.Why it's better: Captures semantic relationships (e.g., "king" - "man" + "woman" ≈ "queen").Limitations: Requires large datasets and pretraining.
Transformer-Based Models (BERT, GPT, T5, etc.)
How it works: Uses deep learning to learn word representations dynamically based on context.Why it's better: Handles word sense disambiguation, long-range dependencies, and polysemy (words with multiple meanings).Limitations: Computationally expensive and requires large amounts of training data.
Latent Semantic Analysis (LSA) & Latent Dirichlet Allocation (LDA)
How it works: Identifies hidden topics in text by analyzing word co-occurrence patterns.Why it's better: Useful for topic modeling and document clustering.Limitations: Less interpretable than embeddings and transformers.
Would you like a practical demo comparing TF-IDF, Word Embeddings, and Transformer-based models?
Stemming vs. LemmatizationFeatureStemmingLemmatizationDefinitionRemoves suffixes to get a root formConverts words to their dictionary formMethodUses simple rules (e.g., "running" → "run")Uses linguistic knowledge (e.g., "better" → "good")AccuracyLess accurate, aggressive truncationMore accurate, considers word meaningSpeedFasterSlowerExample"Caring" → "Car""Caring" → "Care"When to Use Which?Use Stemming when speed is more important than accuracy (e.g., quick text analysis).Use Lemmatization for NLP applications where correct word forms matter (e.g., search engines, chatbots).
Would you like a Python demo comparing stemming and lemmatization on a dataset?
You said:
Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
Can you think of techniques that are better than both BoW and TF-IDF ?
Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.
ChatGPT said:
Why Does TF-IDF Perform Better Than Bag-of-Words (BoW)?
TF-IDF improves upon BoW by considering both term frequency (TF) and inverse document frequency (IDF), which helps distinguish important words from common ones. Here's why it generally leads to better accuracy:
Reduces the Impact of Common Words
BoW treats all words equally, meaning frequently occurring but unimportant words (e.g., "the", "is", "and") get the same importance as meaningful words.TF-IDF penalizes common words by assigning lower weights to those that appear in many documents.
Prioritizes Informative Words
Words that appear frequently in one document but rarely in others get higher weights.This makes TF-IDF better at identifying keywords that define a document’s content.
Improves Discriminative Power
Since TF-IDF reduces noise from common words, it helps classifiers and search engines retrieve more relevant results.In contrast, BoW results in models that focus too much on frequent but unimportant words.
Handles Large Vocabulary More Effectively
BoW creates high-dimensional, sparse feature vectors with many zero values.TF-IDF helps reduce dimensionality by lowering the influence of non-discriminative words.Better Techniques Than BoW and TF-IDF
While TF-IDF improves upon BoW, modern NLP techniques provide even better representations of text by capturing context and semantic meaning:
1. Word Embeddings (Word2Vec, GloVe, FastText)How it works: Maps words to dense, continuous vectors that capture semantic relationships.Why it's better: Unlike BoW/TF-IDF, it understands relationships between words (e.g., "king" - "man" + "woman" ≈ "queen").Limitations: Requires a large dataset for training and may struggle with rare words.2. Transformer-Based Models (BERT, GPT, T5, etc.)How it works: Uses deep learning to generate context-aware word representations.Why it's better: Unlike BoW/TF-IDF, it considers word order, syntax, and polysemy (multiple meanings of words).Limitations: Computationally expensive and requires large-scale training.3. Latent Semantic Analysis (LSA) & Latent Dirichlet Allocation (LDA)How it works: Discovers hidden topics by analyzing word co-occurrence patterns.Why it's better: Useful for topic modeling and document clustering.Limitations: Doesn’t capture word meaning as effectively as word embeddings.
Would you like a Python example comparing TF-IDF, Word Embeddings, and Transformers?
Stemming vs. Lemmatization
Both techniques normalize words to their root form, but they work differently.
FeatureStemmingLemmatizationDefinitionRemoves suffixes to get the word rootConverts words to their dictionary formMethodUses simple rules (e.g., "running" → "run")Uses linguistic knowledge (e.g., "better" → "good")AccuracyLess accurate, aggressive truncationMore accurate, considers word meaningSpeedFasterSlowerExample"Caring" → "Car""Caring" → "Care"When to Use Which?Use Stemming when speed is more important than accuracy (e.g., quick text processing).Use Lemmatization for NLP tasks requiring proper word forms (e.g., search engines, chatbots).
Would you like a Python demo comparing stemming and lemmatization on real text data?

In [ ]:
import pandas as pd
df = pd.read_csv('spam.csv')
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ã¼ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

In [ ]:
df.head(5)

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
len(df)

5572

In [ ]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 2: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2, metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

In [ ]:
# This cell may take some time to run
predicted, y_test = bow_knn()

<ipython-input-2-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]




In [ ]:
# This cell may take some time to run
predicted, y_test = tfidf_knn()

<ipython-input-2-3fc32437a98d>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


KNN with TFIDF accuracy = 98.56502242152466%
Cross Validation Accuracy: 0.97
[0.96837147 0.96769852 0.96363636]


### Questions to Think About and Answer
1. Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
2. Can you think of techniques that are better than both BoW and TF-IDF ?
3. Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.

### Useful Resources for further reading
1. Stemming and Lemmatization: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html
2. TF-IDF and BoW : https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/
3. TF-IDF: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
